In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import sys
sys.path.append("../")

In [2]:
from sprintdl.main import *
from sprintdl.nets import *

In [3]:
device = torch.device('cuda',0)
from torch.nn import init
import torch
import math

# Define required

In [4]:
url = "https://s3.amazonaws.com/fast-ai-imageclas/imagewoof2-160.tgz"

fpath = download_and_check(url, fpath = "/media/hdd/Datasets/", name = "imagewoof")

Already downloaded at /media/hdd/Datasets/imagewoof2-160.tgz


In [5]:
fpath = untar_data(fpath)

Extracted to /media/hdd/Datasets/imagewoof2-160


In [6]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
bs = 256

# Define model and data

In [7]:
lr = 1e-2
pct_start = 0.5
phases = create_phases(pct_start)
sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
sched_mom = combine_scheds(phases, cos_1cycle_anneal(0.95, 0.85, 0.95))

cbfs = [
    partial(AvgStatsCallback,accuracy),
    partial(ParamScheduler, 'lr', sched_lr),
    partial(ParamScheduler, 'mom', sched_mom),
        partial(BatchTransformXCallback, norm_imagenette),
    ProgressCallback,
    Recorder,
#     MixUp,
       partial(CudaCallback, device)]

loss_func=LabelSmoothingCrossEntropy()
arch = partial(xresnet18, c_out=10)
epochs = 5
lr = .4
# opt_func = partial(sgd_mom_opt, wd=0.01)
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)
# opt_func = lamb

# Actual process

In [9]:
il = ImageList.from_files(fpath, tfms=tfms)

In [10]:
il

ImageList (12954 items)
[PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_8554.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_4035.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_13405.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_50879.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_14617.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_15723.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_8595.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_10705.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/ILSVRC2012_val_00002786.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_32024.JPEG')...]
Path: /media/hdd/Datasets/imagewoof2-160

In [11]:
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())

In [12]:
ll

SplitData
Train: LabeledData
x: ImageList (9025 items)
[PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_8554.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_4035.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_13405.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_50879.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_14617.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_15723.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_8595.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_10705.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/ILSVRC2012_val_00002786.JPEG'), PosixPath('/media/hdd/Datasets/imagewoof2-160/train/n02115641/n02115641_32024.JPEG')...]
Path: /media/hdd/Datasets/imagewoof2-160
y:ItemList (9025 items)
[0, 0, 0, 0

In [13]:
data = ll.to_databunch(bs, c_in=3, c_out=10)

# Training

In [14]:
# learn = get_learner(nfs, data, lr, conv_layer, cb_funcs=cbfs)
learn = Learner(arch(), data, loss_func, lr=lr, cb_funcs=cbfs, opt_func=opt_func)

In [15]:
# model_summary(learn, data)

In [16]:
learn.fit(epochs)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.192717,0.224820,2.443135,0.203360,00:15
1,1.992260,0.322105,2.807140,0.242555,00:15
2,1.864577,0.379612,2.529550,0.251209,00:15
3,1.667061,0.468366,1.714755,0.459913,00:15
4,1.445519,0.583380,1.519326,0.548995,00:16


# Save model

In [20]:
sm1 = save_model(learn, "5epoch", fpath)

# New data 

In [8]:
tfms = [make_rgb, ResizeFixed(256), to_byte_tensor, to_float_tensor]
cbfs = [
    partial(AvgStatsCallback,accuracy),
    partial(ParamScheduler, 'lr', sched_lr),
    partial(ParamScheduler, 'mom', sched_mom),
        partial(BatchTransformXCallback, norm_imagenette),
    ProgressCallback,
#     Recorder,
#     MixUp,
       partial(CudaCallback, device)]
bs = 64

In [9]:
il = ImageList.from_files(fpath, tfms=tfms)

In [10]:
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())

In [11]:
data = ll.to_databunch(bs, c_in=3, c_out=10)

In [17]:
# learn = get_learner(nfs, data, lr, conv_layer, cb_funcs=cbfs)
learn = Learner(arch(), data, loss_func, lr=lr, cb_funcs=cbfs, opt_func=opt_func)

In [19]:
adapt_model(learn, data, "/media/hdd/Datasets/imagewoof2-160/models/5epoch")

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [14]:
learn.fit(epochs)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.804654,0.412742,2.172411,0.331637,00:45


KeyboardInterrupt: 